**Note: be sure to add the folder Brown-Datathon-Public-2020 to your drive (you can do this using the dropdown arrow next to the folder in the path header in drive) before running this notebook.**

## Presets

In [0]:
# Make necessary/useful imports here
import pandas as pd
import random
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import matplotlib.cm
from sklearn import metrics
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix

# Set pandas options to not truncate
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', -1)

## Fix for engine=python error. 
# Do not Mount Data 
#from google.colab import drive
#drive.mount("/content/gdrive", force_remount=True)
#root_path = 'gdrive/My Drive/Datathon-2020/Brown-Datathon-Public-2020/Citizens-Home-Financing-Challenge/data/'
# Instead, download and unzip data directly to colab
root_path = 'citizens-home-financing-challenge/'
!wget https://datathon2020.s3.amazonaws.com/datasets/citizens-home-financing-challenge.zip
!unzip citizens-home-financing-challenge.zip


Mounted at /content/gdrive


## Data Dictionaries

In [0]:
# Look over filenames and dimensions of provided and withheld sets
pd.read_csv(root_path + "dictionaries/datathon_summary.csv")

,File Name,# Rows,# Fields,Summary of Fields,Notes
0,zip9_coded_201904_pv,6009259,53,April snapshot of Experian data,"""pv"" stands for ""provided""; students will have this set"
1,zip9_coded_201905_pv,6009259,53,May snapshot of Experian data,"""pv"" stands for ""provided""; students will have this set"
2,zip9_coded_201906_pv,6009259,53,June snapshot of Experian data,"""pv"" stands for ""provided""; students will have this set"
3,zip9_coded_201907_pv,6009259,53,July snapshot of Experian data,"""pv"" stands for ""provided""; students will have this set"
4,zip9_coded_201908_pv,6009259,53,August snapshot of Experian data,"""pv"" stands for ""provided""; students will have this set"
5,zip9_coded_201909_pv,6009259,53,September snapshot of Experian data,"""pv"" stands for ""provided""; students will have this set"
6,zip9_demographics_coded_pv,6009259,7,Demographic and homebuyer data,"""pv"" stands for ""provided""; students will have this set"
7,zip9_coded_201904_wh,667418,53,April snapshot of Experian data,"""wh"" stands for ""withheld"""
8,zip9_coded_201905_wh,667418,53,May snapshot of Experian data,"""wh"" stands for ""withheld"""
9,zip9_coded_201906_wh,667418,53,June snapshot of Experian data,"""wh"" stands for ""withheld"""


In [0]:
# Of the demographics/homebuyer data, observe provided field descriptions
pd.read_csv(root_path + "dictionaries/zip9_demographics_coded_dict.csv")

,Index,Variable,Type,Description,Notes
0,1,zip5,Char,Five digit zip code,NaN
1,2,zip9_code,Num,Numerical key matching to a zip9 region,NaN
2,3,age,Num,Average age in the zip9 region,May skew high since we only look at credit-active adults
3,4,household_count,Num,Number of households in the zip9 region,This dataset excludes any row with four or less households
4,5,person_count,Num,Number of people in the zip9 region,NaN
5,6,homebuyers,Num,Number of people in the zip9 region who bought a home between October - December 2019,NaN
6,7,first_homebuyers,Num,Number of people in the zip9 region who bought their first home home between October - December 2019,NaN


In [0]:
# Of the credit bureau data, observe provided field descriptions
pd.read_csv(root_path + "dictionaries/zip9_coded_20190X_dict.csv")

,Index,Variable,Type,Description,Notes
0,1,zip5,Char,Five digit zip code,NaN
1,2,zip9_code,Num,Numerical key matching to a zip9 region,NaN
2,3,bankcard_limit,Num,Average bank card limit per cardholder in the zip9 region,For holders of more than one bank card the bank card limit is the sum of limits
3,4,bankcard_balance,Num,Average bank card balance per cardholder in the zip9 region,For holders of more than one bank card the bank card balance is the sum of balances
4,5,bankcard_trades,Num,Average number of open bank cards per person in the zip9 region,NaN
5,6,bankcard_util,Num,Average ratio of bank card balance to limit per cardholder in the zip9 region,NaN
6,7,total_revolving_limit,Num,Average sum of bank card/other revolving/installment limits per person in the zip9 region,NaN
7,8,total_revolving_balance,Num,Average sum of bank card/other revolving/installment balances per person in the zip9 region,NaN
8,9,total_revolving_trades,Num,Average number of open bank card/other revolving/installment trades per person in the zip9 region,NaN
9,10,total_revolving_util,Num,Average ratio of total revolving balance to limit per person in the zip9 region,NaN


## Looking at Data Samples

In [0]:
# Here, we look at a random sample of the demographic/homebuyer data and of the September credit bureau
#  snapshot. The sample size in this template is kept quite small so it runs quickly as an example, so
#  please adjust it to your preferences/requirements

# Pandas can have trouble dealing with moderately large data, here's a sampling example
n = 6009259 # number of records in file
s = 60000 # sample size (can/should be changed to your preference)

skip_list = sorted(random.sample(range(1,n+1),n-s))
sep_df = pd.read_csv(root_path + "zip9_coded_201908_pv.csv", skiprows=skip_list, dtype={'zip5': str})
demo_df = pd.read_csv(root_path + "zip9_demographics_coded_pv.csv", skiprows=skip_list, dtype={'zip5': str})

# Take a look at a few examples in the September credit bureau snapshot
sep_df.head()

# TIP: Many Python libraries perform better with the magnitude of data provided, such as PySpark or Dask.
#  Theses packages are well-documented online. We leave it up to you to decide how to handle the moderately
#  large data. Alternatively, you can also read by chunks via pandas, although there are limitations to it

ParserError: ignored

In [0]:
# Check the datatypes of the fields in the September credit bureau snapshot
# These should reflect the fields seen in the data dictionary
sep_df.dtypes

In [0]:
# Take a look at a few examples in the demographic/homebuyer data
demo_df.head()

In [0]:
# Check the datatypes of the fields in the demographic/homebuyer data
# These should reflect the fields seen in the data dictionary; particularly, homebuyers and first_homebuyers
#  are of the most interest, as they are the targets
demo_df.dtypes

## Basic Exploration of Data Samples

In [0]:
# Exploratory data analysis is imperative to understanding datasets/target fields, assessing which fields
#  may be necessary in predictive analysis, which types of models are viable, and what other pre-processing
#  techniques may be necessary prior to training. Here is a very rudimentary start to exploring

# Look at summary of the fields in the September credit bureau snapshot
# Certain fields will have missing values, think about how to handle these
sep_df.describe()

In [0]:
# Look at summary of the fields in the demographic/homebuyer data
demo_df.describe()

# TIP: Note that the targets (homebuyers/first homebuyers) have very low average values; this means that not
#  many zip9 have non-zero values for these fields, which may become an issue if you train as is, or use a
#  model that cannot accommodate this bias. Some options to deal with this, which you may already be aware of,
#  involve selective sampling from the raw data to ensure a more balanced target distribution, and algorithms
#  like SMOTE (Synthetic Minority Oversampling Technique) that will synthesize minority examples. Failure to
#  account for this may lead to predictions of only zero

In [0]:
# Visualize the distribution of the homebuyer field on a log scale
figure, ax = plt.subplots()
plt.title('Logarthmic Homebuyer Distribution', fontsize=15)
plt.xlabel('Number of Homebuyers', fontsize=10)
plt.ylabel('Number of Zip9 Regions', fontsize=10)
ax.set_yscale('log')
demo_df['homebuyers'].value_counts().plot(ax=ax, kind='bar')
demo_df['homebuyers'].value_counts()

In [0]:
# Many of the fields are kept separate from the targets, so having a merged set would be valuable
sep_demo_merge = sep_df.merge(demo_df,
                              how='inner',
                              on='zip9_code',
                              suffixes=('_sep','_demo'),
                              validate='one_to_one')

In [0]:
# Having the merged set allows us to look at target-based field distributions
sep_demo_merge.groupby('homebuyers').mean()

In [0]:
# Having the merged set also allows us to graph how a field may relate to the likelihood of buying a home
test = sep_demo_merge
test['bankcard_balance_decile'] = pd.qcut(sep_demo_merge['bankcard_balance'], q=10)

test = test.pivot_table(columns = ['bankcard_balance_decile'], values = ['homebuyers'])
test = test.transpose()
test['bankcard_balance_decile'] = test.index

test.plot(kind='bar')
plt.title('Relation between Bankcard Balance and Homebuyer Count')
plt.xlabel('Bankcard Balance Decile', fontsize=10)
plt.ylabel('Proportion of Homebuyers', fontsize=10)

## Baseline Model

### Pre-processing

In [0]:
# Before modeling, traditionally you will want to pre-process the data by cleaning, normalizing, imputing,
#  or otherwise converting inputs into the format you want your model to receive. In this case, we are
#  providing a baseline model that will perform this, but not to a high standard

# For simplicity, we will make the (harsh) assumption that the most recent snapshot (September) and
#  demographic/homebuyer information are all that matter for predicting, then get rid of fields with
#  a majority of values missing, which we know from earlier exploration
majority_missing = ['mortgage2_limit','mortgage2_balance','mortgage3_limit','mortgage3_balance',
                    'mortgage4_limit','mortgage4_balance','mortgage5_limit','mortgage5_balance',
                    'homeequity1_limit','homeequity1_balance','homeequity2_limit','homeequity2_balance',
                    'homeequity3_limit','homeequity3_balance','homeequity4_limit','homeequity4_balance',
                    'homeequity5_limit','homeequity5_balance','total_homeequity_limit',
                    'total_homeequity_balance','homeequity1_loan_to_value']
# Remove exploration, secondary target, and other extra fields
other_removals = ['bankcard_balance_decile','first_homebuyers','zip5_sep','zip5_demo','zip9_code'] 
sep_demo_merge_vars = sep_demo_merge.columns.values.tolist()
to_keep = [i for i in sep_demo_merge_vars if i not in majority_missing]
to_keep = [i for i in to_keep if i not in other_removals]
data_final = sep_demo_merge[to_keep]

# TIP: An important piece to consider when preparing to model is which features to select. Some may
#  have little to no predictive ability, and others may be too sparse to be useful. Algorithms such as RFE
#  (Recursive Feature Elimination) can help you decide how to eliminate bad features

In [0]:
# This leaves only fields that are fully populated, and seven fields that are mostly populated
# We will (again, inadvisably) assume that the seven fields with missing values can be imputed with zeroes
# These fields are bankcard_util, total_revolving_util, mortgage1_limit, mortgage1_balance,
#  total_mortgage_limit, total_mortgage_balance, mortgage1_loan_to_value
data_final = data_final.fillna(value=0)

In [0]:
# Our basic model will simply predict the presence of homebuyers in a zip9 region, a highly related problem
data_final['homebuyers'][data_final['homebuyers'] != 0] = 1 # you can ignore the warning
data_final['homebuyers'].value_counts()

In [0]:
# A way to split into observations and labels
X = data_final.loc[:, data_final.columns != 'homebuyers']
y = data_final.loc[:, data_final.columns == 'homebuyers']

print(X.shape)
print(y.shape)
print(X.columns.values)
print(y.columns.values)

In [0]:
# A way to split into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.16, random_state=0)

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

### Modeling

In [0]:
# Input resulting sets into very basic logistic regression model
logreg = LogisticRegression(solver='lbfgs',max_iter=1000,tol=0.001)
logreg.fit(X_train, y_train.values.ravel())

In [0]:
# Use the baseline model to make predictions

# As mentioned earlier, one of the issues with feeding in data with a vastly imbalanced label class is 
#  that predictions can tend towards zero and still maintain good raw accuracy (correct/total). In logistic
#  regression, one can more or less get past this by setting a custom threshold on logits
threshold = 0.13 # can be tweaked to see how it would affect raw accuracy

y_pred_probs = pd.DataFrame(logreg.predict_proba(X_test)[:,1])
y_pred = y_pred_probs.applymap(lambda x: 1 if x > threshold else 0)
y_pred = np.reshape(np.array(y_pred), (y_pred.shape[0],))

conf_matrix = confusion_matrix(y_test, y_pred)
correct = conf_matrix[0,0] + conf_matrix[1,1]
incorrect = conf_matrix[0,1] + conf_matrix[1,0]
total = correct + incorrect

print(conf_matrix)
print("Number Correct: " + str(correct))
print("Number Incorrect: " + str(incorrect))
print("Raw Accuracy: " + str(correct/total))

### Loss Calculation

In [0]:
# As with most predictors, there are options for which loss function to use; since your predictions represent
#  number of homebuyers in a zip9, the loss calculation could be as simple as average square difference;
#  however, this overlooks the fact that the label imbalance issue is likely to produce models that predict
#  zeroes. We can get around this by having a second loss function that emphasizes loss due to underprediction

# This function expects two numpy vectors of equal size, and outputs average square difference
def loss_function_1(y_pred, y_labels):
    size = y_pred.size
    differences = y_pred - y_labels
    differences_square = differences*differences
    differences_square_sum = np.sum(differences_square)
    
    return differences_square_sum/size

# Likewise, expects two numpy vectors of equal size, and outputs average scaled square difference
def loss_function_2(y_pred, y_labels):
    size = y_pred.size
    differences = y_pred - y_labels
    
    # If the prediction is smaller than the label, then we have underpredicted, scale this up
    under = np.where(differences < 0, differences, 0)
    over = np.where(differences > 0, differences, 0)
    under_square_scaled = under*under*10
    over_square = over*over
    square_sum = np.sum(under_square_scaled) + np.sum(over_square)
    
    return square_sum/size

# TIP: On a test set with a target distribution similar to the raw data, the number of zeroes predicted 
#  will likely skew high, since that would correlate well with raw accuracy. Loss function two acts similarly
#  to the first loss function, but with an emphasis on false underpredictions. A good model should perform
#  well under both metrics

In [0]:
# Let's see how we did
print("Loss 1:", loss_function_1(y_pred, y_test.values.ravel()))
print("Loss 2:", loss_function_2(y_pred, y_test.values.ravel()))

In [0]:
# ...A pretty low bar. As it turns out, a model predicting only zeroes for number of homebuyers on this
#  particular sample testing set would have about Loss 1 of ~0.03177, and Loss 2 of 0.31771

# On another sample that is designed (either by selective sampling, oversampling, etc.) to be more balanced,
#  one should aim to generally have Loss 1 under 0.1 and Loss 2 under 0.3. Failing this, your model should
#  definitely make better predictions on balanced data by both loss metrics than the baseline model above

# Good luck!